In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline

In [ ]:
# Load data
df = pd.read_csv("/Users/jingang/Desktop/Capstone/SPY.csv")
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
Price = df.loc[:,"Adj Close"]

In [ ]:
def divide_period(window_length = 63, series = Price):
    """
    return the vector that divides the series into windows of fixed size
    """

    # define number of windows
    num_windows = len(series) / window_length

    # divide the Price into num_windows
    window_index = np.arange(0,(1+num_windows)*window_length,window_length)
    return window_index

In [ ]:
# find the date
window_index = divide_period()
price_index = Price.index[window_index[1:]]

In [ ]:
# compute_ema (the middle band)
def compute_ema(series=Price, short_window=5):
    return series.ewm(span=short_window,adjust=False).mean()

In [ ]:
# compute the bollinger_band
def compute_bollinger_band(series=Price, span=20, num_std=1.2):
    bollinger_band_middle = Price.ewm(span=span).mean()
    rolling_std = Price.ewm(com=9.5).std()
    bollinger_band_upper = bollinger_band_middle+rolling_std*num_std
    bollinger_band_lower = bollinger_band_middle-rolling_std*num_std
    return bollinger_band_upper,bollinger_band_middle, bollinger_band_lower

In [ ]:
ema_26 = Price.ewm(26).mean()
ema_12 = Price.ewm(12).mean()
MACD_line = ema_12 - ema_26
Signal_line = MACD_line.ewm(9).mean()
hist = MACD_line - Signal_line
df = pd.DataFrame({'Price':Price,'ema_12': ema_12, 'ema_26':ema_26,'MACD_line':MACD_line, 'Signal_line':Signal_line, 'Hist':hist},
               columns=['Price', 'ema_12','ema_26','MACD_line','Signal_line', 'Hist'])

_, ax1 = plt.subplots(figsize=(14,8))
ax1.plot(df.Price[200:400], c='r', label='price')
ax1.legend(loc=2)
ax1.grid(True)
ax2 = ax1.twinx()
ax2.plot(df.MACD_line[200:400], c='g', label='MACD')
ax2.plot(df.Signal_line[200:400], c='b', label='Signal_line')
ax2.plot(df.Hist[200:400], c='black', label='hist')
ax2.legend(loc=1)

In [ ]:
# MACD strategy

def momentum_strategy(series=Price, short = , long = , num_std=0.1, market_regime=1, window_length = 63):
    """
    for each period, check if momentum strategy's PNL is + or -
    return a nx1 vector Of gain or loss
    """
    # construct MACD
    # MACD params 26, 12, 9
    ema_26 = Price.ewm(26).mean()
    ema_12 = Price.ewm(12).mean()
    MACD_line = ema_12 - ema_26
    Singal_line = MACD_line.ewm(9).mean()
    MACD_hist = MACD_line - Signal_line
    
    # Momentum signal 
    
    
    
    

    window_index = divide_period(window_length = window_length, series = series[1:])
    
    # result vector
    res = np.zeros(len(window_index) - 1)
    
    # trade count
    trade_count = np.zeros(len(window_index) - 1)
    
    # momentum strategy
    # for every periods
    for i in range(len(window_index)-1):
        current_price = series[window_index[i]:window_index[i+1]]
        current_period = series.ewm(span=short_window,adjust=False).mean()[window_index[i]:window_index[i+1]]
        current_upper = bb_upper[window_index[i]:window_index[i+1]]
        current_middle = bb_middle[window_index[i]:window_index[i+1]]
        current_lower = bb_lower[window_index[i]:window_index[i+1]]
        
        # notice the the indicators are designed such that there is no chance of them overlap
        # price cross middle from below
        long_indicator = np.all([current_period > current_upper, current_period.shift(1) < current_upper.shift(1)], axis=0)
        
        # price cross middle from above
        close_long_indicator = np.all([current_period < current_middle, current_period.shift(1) > current_middle.shift(1)], axis=0)
    
        # price cross lower from above
        short_indicator = np.all([current_period < current_lower, current_period.shift(1) > current_lower.shift(1)], axis=0)
        
        # price cross middle from below
        close_short_indicator = np.all([current_period > current_middle, current_period.shift(1) < current_middle.shift(1)], axis=0)
        
        
        
        
        # make the strategy
        # if eixst position, close it anyway at end of the period
        position = 0
        pnl = 0
        pre_price = 0
        count = 0
        
        for j in range(window_length):

            # long indicator is activated
            if long_indicator[j] == True and position == 0:
                position = 1
                pre_price = current_price[j]
                
            # close long indicator is activated
            if close_long_indicator[j] == True and position == 1:
                position = 0
                pnl += current_price[j] - pre_price
                pre_price = 0
                count += 1
            
            # short indicator is activated
            if short_indicator[j] == True and position == 0:
                position = -1
                pre_price = current_price[j]
            
            # close short indicator is activated
            if close_short_indicator[j] == True and position == -1:
                position = 0
                pnl += pre_price - current_price[j]
                pre_price = 0
                count += 1
                
        # close any existing position
        if position != 0:
            if position == 1:
                pnl += current_price[window_length-1] - pre_price
            if position == -1:
                pnl += pre_price - current_price[window_length-1]
                
        res[i] = pnl 
        trade_count[i] = count
        
    return res, trade_count

In [ ]:
def mean_reversion_strategy(series=Price, long_window=10, num_std=1.2, market_regime=1, window_length = 63):
    """
    for each period, check if mean reversion strategy's PNL is + or -
    return a nx1 vector Of gain or loss
    """
    # EMA indicator
    bb_upper, bb_middle, bb_lower = compute_bollinger_band(series, long_window, num_std)
    
    # periods
    window_index = divide_period(window_length = window_length, series = series[1:])
    
    # result vector
    res = np.zeros(len(window_index) - 1)
    
    # trade count
    trade_count = np.zeros(len(window_index) - 1)
    
    # mean reversion strategy
    # for every periods
    """
    a=b=c=d=[]
    e=f=g=h=[]
    """ 
    
    for i in range(len(window_index)-1):

        current_period = series[window_index[i]:window_index[i+1]]
        current_upper = bb_upper[window_index[i]:window_index[i+1]]
        current_middle = bb_middle[window_index[i]:window_index[i+1]]
        current_lower = bb_lower[window_index[i]:window_index[i+1]]
        
        # notice the the indicators are designed such that there is no chance of them overlap
        # price cross lower from above
        long_indicator = np.all([current_period < current_lower, current_period.shift(1) > current_lower.shift(1)], axis=0)
        
        # price cross middle from below
        close_long_indicator = np.all([current_period > current_middle, current_period.shift(1) < current_middle.shift(1)], axis=0)
    
        # price cross middle from above
        close_short_indicator = np.all([current_period < current_middle, current_period.shift(1) > current_middle.shift(1)], axis=0)
        
        # price cross upper from below
        short_indicator = np.all([current_period > current_upper, current_period.shift(1) < current_upper.shift(1)], axis=0)
        
        
        ############
        """
        if i == 1:
            a=current_period
            b=current_upper
            c=current_middle
            d=current_lower
            
            e=long_indicator
            f=close_long_indicator
            g=close_short_indicator
            h=short_indicator
        """    
        ############
        # make the strategy
        # if eixst position, close it anyway at end of the period
        position = 0
        pnl = 0
        pre_price = 0
        count = 0
        for j in range(window_length):
                
            # long indicator is activated
            if long_indicator[j] == True and position == 0:
                position = 1
                pre_price = current_period[j]
                
            # close long indicator is activated
            if close_long_indicator[j] == True and position == 1:
                position = 0
                pnl += current_period[j] - pre_price
                pre_price = 0
                count += 1
            
            # short indicator is activated
            if short_indicator[j] == True and position == 0:
                position =  -1
                pre_price = current_period[j]
            
            # close short indicator is activated
            if close_short_indicator[j] == True and position == -1:
                position = 0
                pnl += pre_price - current_period[j]
                pre_price = 0
                count += 1
                
        # close any existing position
        if position != 0:
            if position == 1:
                pnl += current_period[window_length-1] - pre_price
            if position == -1:
                pnl += pre_price - current_period[window_length-1]
                
        res[i] = pnl 
        trade_count[i] = count
        
    return res, trade_count

In [ ]:
from matplotlib.ticker import *


return_MR, trade_count_MR=mean_reversion_strategy();
plt.hist(return_MR, bins=20);
MR_series = pd.DataFrame({'trasanction':trade_count_MR, 'time':price_index.values,'return':return_MR})

MR_series = MR_series.set_index("time")
MR_series.plot.bar(subplots=True, figsize=(12,10))

In [ ]:
return_MO, trade_count_MO=momentum_strategy();
plt.hist(return_MO, bins=20);
MO_series = pd.DataFrame({'trasanction':trade_count_MO, 'time':price_index.values, 'return':return_MO})
MO_series = MO_series.set_index("time")
MO_series.plot.bar(subplots=True, figsize=(12,10))

In [ ]:
b_return_MR = np.where(return_MR>0, 1, 0)
b_return_MO = np.where(return_MO>0, 1, 0)
regime = zip(b_return_MR,b_return_MO )
r1 = (0,0)
r2 = (0,1)
r3 = (1,0)
r4 = (1,1)
regime_indicator = np.empty_like(b_return_MO)
for i in range(len(regime)):
    if regime[i] == r1:
        regime_indicator[i] = 0
    elif regime[i] == r2:
        regime_indicator[i] = 1
    elif regime[i] == r3:
        regime_indicator[i] = 2
    elif regime[i] == r4:
        regime_indicator[i] = 3
regime_df = pd.DataFrame({'frame':regime_indicator, 'time':MO_series.index.values}) 
regime_df.set_index("time");